In [2]:
import mysql.connector
from mysql.connector import Error

In [28]:
# https://docs.docker.com/docker-for-mac/networking/#use-cases-and-workarounds

connection_config = {
    'user':'bormanjo',
    'password':'Brothers1883!1',
    'host':'host.docker.internal',
    'database':'portfolio_optimizer_db',
    'raise_on_warnings': True,
    'use_pure': False,
    'autocommit' : True,  
    'pool_size' : 5
}

In [55]:
try:
    connection = mysql.connector.connect(**connection_config)
    if connection.is_connected():
        db_info = connection.get_server_info()
        print("Succesfully Connected to MySQL database. MySQL Server version on ",db_info)
except Error as e:
    print ("Error while connecting to MySQL", e)

Succesfully Connected to MySQL database. MySQL Server version on  8.0.12


In [70]:
# Get the cursor object from the DB connection
cursor = connection.cursor(buffered=True)

In [58]:
def create_database(cursor, DB_NAME):
    try:
        cursor.execute(
        "CREATE DATABASE {}".format(DB_NAME))
    except Error as e:
        print("Failed creating database: {}".format(e))
        exit(1)

In [6]:
TABLES = {}

In [7]:
TABLES['portfolio_weights_eod'] = (
    "CREATE TABLE `portfolio_weights_eod` ("
    "  `date` date NOT NULL,"
    "  `ticker` varchar(5) NOT NULL,"
    "  `weight` float NOT NULL,"
    "  `amount` float NOT NULL,"
    "  `close` float NOT NULL,"
    "  `shares` integer NOT NULL,"
    "  PRIMARY KEY (`date`, `ticker`),"
    "  KEY `date` (`date`), KEY `ticker` (`ticker`)"
    ") ENGINE=InnoDB")

TABLES['portfolio_weights_current'] = (
    "CREATE TABLE `portfolio_weights_current` ("
    "  `ticker` varchar(5) NOT NULL,"
    "  `weight` float NOT NULL,"
    "  `amount` float NOT NULL,"
    "  `close` float NOT NULL,"
    "  `shares` integer NOT NULL,"
    "  PRIMARY KEY (`ticker`),"
    "  KEY `ticker` (`ticker`)"
    ") ENGINE=InnoDB")

TABLES['ohlcv'] = (
    "CREATE TABLE `ohlcv` ("
    "  `date` date NOT NULL,"
    "  `ticker` varchar(5) NOT NULL,"
    "  `open` float NOT NULL,"
    "  `high` float NOT NULL,"
    "  `low` float NOT NULL,"
    "  `close` float NOT NULL,"
    "  `volume` float NOT NULL,"
    "  PRIMARY KEY (`date`, `ticker`),"
    "  KEY `date` (`date`), KEY `ticker` (`ticker`)"
    ") ENGINE=InnoDB")

In [31]:

for tbl_name, q in TABLES.items():
    try:
        print("Creating table {}:".format(tbl_name), end='')
        cursor.execute(q)
    except Error as e:
        if e.errno == mysql.connector.errorcode.ER_TABLE_EXISTS_ERROR:
            print("Table {} already exists".format(tbl_name))
        else:
            print(e.msg)
    else:
        print("OK")

Creating table portfolio_weights_eod:Table portfolio_weights_eod already exists
Creating table portfolio_weights_current:Table portfolio_weights_current already exists
Creating table ohlcv:Table ohlcv already exists


In [67]:
def get_tables(cursor):
    try:
        cursor.execute("USE portfolio_optimizer_db;")
        cursor.execute("SHOW TABLES;")
        result = cursor.fetchall()
        return result
    except Error as e:
        print("Failed to get tables: {}".format(e))
        exit(1)


In [68]:
print(get_tables(cursor))

[('ohlcv',), ('portfolio_weights_current',), ('portfolio_weights_eod',)]


In [65]:
print(result)

[('ohlcv',), ('portfolio_weights_current',), ('portfolio_weights_eod',)]
